In [ ]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt
import pandas as pd
from baseline_utils import generate_grid_coordinates, generate_grid_values_from_sourceloc, generate_sourceloc_from_coordinates, add_anomaly
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras import layers
from keras.models import Model
#from tensorflow.keras.utils import np_utils
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.datasets import mnist
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from tensorflow.keras.layers import Dense, Input, Flatten,Reshape, LeakyReLU as LR,Activation, Dropout
from tensorflow.keras.models import Model, Sequential
import argparse
import os
from autoencoder_model import autoencoder_model
from cnn_model import cnn_model
from mean_reconstruction import mean_reconstruction
from plot_grid import plot_grid
plt.rcParams["figure.figsize"] = (13,7)

In [ ]:
test_sample = np.load('test_sample_obadah.npy')
size = 100

In [ ]:
def plot_grid(source_locations,array,size):
    arr = np.copy(array) 
    fig, ax = plt.subplots()
    ax.grid(which='major', color='black', linestyle='-', linewidth=1)
    # To display the values
    ax.set_xlim([0,size**2])
    ax.set_ylim([0,size**2])
    for [i,j], z in zip(source_locations,arr.ravel()):
        ax.text(i+0.5, j+0.5, '{:0.1f}'.format(z), ha='center', va='center')
    plt.xticks(np.arange(0, size**2, size))
    plt.yticks(np.arange(0, size**2, size))
    plt.show()
    
def generate_grid_coordinates(size):
    grid_coordinates = np.zeros((size,size,2))
    for i in range(size):
        for j in range(size):
            grid_coordinates[i][j][0] = ((i+1) * size) - size/2
            grid_coordinates[i][j][1] = ((j+1) * size) - size/2
    return grid_coordinates

In [ ]:
grid_coordinates = generate_grid_coordinates(size) ## generate sensor locations for training (each sensor placed at center of each grid)
sensor_locations = grid_coordinates.reshape(size**2, 2) ## sensor locations

In [ ]:
plot_grid(sensor_locations, test_sample[0], size)

In [ ]:
zero_loc = np.where(test_sample[0] == 0)

In [ ]:
zero_loc[0]

In [ ]:
test_sample[0].shape

In [ ]:
np.count_nonzero(test_sample[0]==0)

In [ ]:
t1 = test_sample[0]

In [ ]:
t1[t1==0] = np.nan

In [ ]:
window_size = 10
while(np.isnan(t1).any()):
    t1 = mean_reconstruction(t1, window_size)
    window_size += 1

In [ ]:
t1

In [ ]:
    print("Loading pretrained models")
    autoencoder = load_model("saved_models/cdae_pretrained", custom_objects=None,compile=True)
    cnn = load_model("saved_models/cnn_pretrained")
    print("Pretrained models successfully loaded")

In [ ]:
test_autoencoder_output = autoencoder.predict(t1.reshape(1, size, size, 1))

In [ ]:
    test_cnn_output = cnn.predict(test_autoencoder_output)
    pred = test_cnn_output.reshape(size,size)
    print("Predicted source location: ", np.where(pred == np.amax(pred)))
    plot_grid(sensor_locations, test_cnn_output[0], size)

In [ ]:
np.argmax(test_autoencoder_output.reshape(100,100))

In [ ]:
test_autoencoder_output.shape

In [ ]:
test_autoencoder_output.reshape(100,100)[3][83]

In [ ]:
test_autoencoder_output.shape

In [ ]:
np.where(t1 == np.amax(t1))

In [ ]:
np.where(test_sample[1] == np.amax(test_sample[1]))

In [ ]:
print("Loading pretrained models")
autoencoder = load_model("saved_models/cdae_pretrained", custom_objects=None,compile=True)
cnn = load_model("saved_models/cnn_pretrained")
print("Pretrained models successfully loaded") 

In [ ]:
loc_error = 0
for samples in np.copy(test_sample):
    window_size = 10
    source_loc = np.where(samples == np.amax(samples))
    samples[samples==0] = np.nan
    while(np.isnan(samples).any()):
        samples = mean_reconstruction(samples, window_size)
        window_size += 1
    
    
    test_autoencoder_output = autoencoder.predict(samples.reshape(1, size, size, 1))
    test_cnn_output = cnn.predict(test_autoencoder_output)
    pred = test_cnn_output.reshape(size,size)
    print("Predicted source location: ", np.where(pred == np.amax(pred)))
    print("Actual source location: ", source_loc)
    print("Localization Error: ", math.dist(source_loc, np.where(pred == np.amax(pred))))
    loc_error += math.dist(source_loc, np.where(pred == np.amax(pred)))

print("The average localization error is: ", loc_error/100)
    

In [ ]:
np.where(test_sample[3] == np.amax(test_sample[3]))

In [ ]:
samples

In [ ]:
    size = 100
    source_intensity = 10000000
    detector_area = 0.02
    efficiency = 1
    print("Initializing test aoi with 200 sensors")
    test_source_location = np.random.rand(1, 2) * (size**2)
    print("Source Location: ", test_source_location)
    test_sensor_locations = np.random.rand(200, 2) * (size**2)
    print(test_sensor_locations.shape)
    test_aoi = np.empty([size,size])
    for loc in test_sensor_locations:
        i = int(loc[0] / 100)
        j = int(loc[1] / 100)
        test_aoi[i][j] = (source_intensity * detector_area * efficiency) / (math.dist(loc, test_source_location.reshape(2))**2)
        
    ## Mean reconstruction
    window_size = 10
    while(np.isnan(test_aoi).any()):
        test_aoi = mean_reconstruction(test_aoi, window_size)
        window_size += 1
        
    print("Loading pretrained models")
    autoencoder = load_model("saved_models/cdae_pretrained", custom_objects=None,compile=True)
    cnn = load_model("saved_models/cnn_pretrained")
    print("Pretrained models successfully loaded")
    
    ## Anomaly correction using pretrained autoencoder
    test_autoencoder_output = autoencoder.predict(test_aoi.reshape(1, size, size, 1))
    
    ## Threshold setting
    
    
    ## Localization using pretrained CNN
    test_cnn_output = cnn.predict(test_autoencoder_output)
    pred = test_cnn_output.reshape(size,size)
    print("Predicted source location: ", np.where(pred == np.amax(pred)))
    plot_grid(sensor_locations, test_cnn_output[0], size)
    